## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-15-22-50-57 +0000,nyt,Boy and Girl Face Rape and Attempted Murder Ch...,https://www.nytimes.com/2025/10/15/us/ohio-chi...
1,2025-10-15-22-50-54 +0000,nyt,A Repressive Russian Ally Feels a Mysterious T...,https://www.nytimes.com/2025/10/15/world/europ...
2,2025-10-15-22-46-00 +0000,wsj,Supreme Court Questions Use of Race in Drawing...,https://www.wsj.com/us-news/law/supreme-court-...
3,2025-10-15-22-45-32 +0000,nypost,California city severely restricts self-checko...,https://nypost.com/2025/10/15/us-news/long-bea...
4,2025-10-15-22-44-33 +0000,nypost,"Hundreds of off-limits, hidden NYC sites to op...",https://nypost.com/2025/10/15/us-news/hundreds...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
66,trump,55
105,new,20
74,hamas,20
97,he,19
287,gaza,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
303,2025-10-15-09-30-00 +0000,wsj,Governors across 15 states are forming a new p...,https://www.wsj.com/health/healthcare/rfk-jr-h...,130
98,2025-10-15-19-58-31 +0000,latimes,California judge halts Trump federal job cuts ...,https://www.latimes.com/politics/story/2025-10...,117
45,2025-10-15-21-22-30 +0000,nyt,Judge Temporarily Blocks Trump From Firing Gov...,https://www.nytimes.com/2025/10/15/us/politics...,113
22,2025-10-15-22-10-23 +0000,nypost,Dems hand out vile ‘Is he dead yet?’ bracelets...,https://nypost.com/2025/10/15/us-news/winnebag...,108
220,2025-10-15-14-21-58 +0000,nyt,Trump Says He May Give Tomahawks to Ukraine. I...,https://www.nytimes.com/2025/10/14/us/politics...,105


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
303,130,2025-10-15-09-30-00 +0000,wsj,Governors across 15 states are forming a new p...,https://www.wsj.com/health/healthcare/rfk-jr-h...
214,75,2025-10-15-14-38-01 +0000,nypost,"Family of kidnapped IDF soldier, 18, only foun...",https://nypost.com/2025/10/15/world-news/idf-s...
189,59,2025-10-15-16-07-50 +0000,nypost,US student group calls for ‘death’ to Zionist ...,https://nypost.com/2025/10/15/world-news/stude...
45,58,2025-10-15-21-22-30 +0000,nyt,Judge Temporarily Blocks Trump From Firing Gov...,https://www.nytimes.com/2025/10/15/us/politics...
238,51,2025-10-15-13-12-06 +0000,nyt,The case will test what is left of the Voting ...,https://www.nytimes.com/live/2025/10/15/us/sup...
365,46,2025-10-15-00-23-28 +0000,nypost,California gov candidate Katie Porter refuses ...,https://nypost.com/2025/10/14/us-news/californ...
293,44,2025-10-15-10-00-00 +0000,nypost,Dr. Keith Ablow tells ‘Pod Force One’ how he b...,https://nypost.com/2025/10/15/us-news/dr-keith...
52,39,2025-10-15-21-09-00 +0000,wsj,SL Green’s deal to buy the 36-story building i...,https://www.wsj.com/real-estate/commercial/sl-...
118,37,2025-10-15-19-28-40 +0000,nypost,Supreme Court seems set to restrict race-based...,https://nypost.com/2025/10/15/us-news/supreme-...
132,36,2025-10-15-18-37-00 +0000,wsj,Lower U.S. Tariffs Would Boost India Growth Ou...,https://www.wsj.com/articles/lower-u-s-tariffs...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
